In [2]:
!pip install transformers
!pip install bitsandbytes-cuda111
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 52.7 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 12.0 MB/s eta 0:00:0000:0100:01


In [4]:
import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
import accelerate
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [5]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [6]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [7]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [8]:
import accelerate

In [9]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

In [10]:
import pandas as pd
import os

In [14]:
import pandas as pd
import os

# Load the prompts
df = pd.read_csv("/kaggle/input/prompts/prompts_pubmed.csv")
prompts = df['Prompt'].tolist()

# Construct the response file path
response_csv_path = os.path.join("/kaggle/working/","gpt-j1x_responses.csv")

# Check if the response file already exists
if os.path.exists(response_csv_path):
    # Load the existing responses
    existing_responses_df = pd.read_csv(response_csv_path)

    # Determine the starting point based on the number of existing responses
    start = len(existing_responses_df)
else:
    start = 0

for i in range(start, len(prompts)):
    # Encode the prompt
    input_ids = tokenizer.encode(prompts[i], return_tensors="pt").to(device)

    # Generate a response
    output = gpt.generate(
            input_ids,
            attention_mask=torch.ones_like(input_ids),  # Set all positions to 1 (i.e., no padding)
            pad_token_id=tokenizer.eos_token_id,  # Use the EOS token as the PAD token
            do_sample=True,
            max_length=1024, 
        )

    # Calculate the number of tokens in the prompt
    prompt_length = input_ids.shape[-1]

    # Decode only the response, excluding the prompt
    response = tokenizer.decode(output[0, prompt_length:], skip_special_tokens=True)

    # Save the prompt and response to a DataFrame
    response_df = pd.DataFrame({
        'Prompt': [prompts[i]],
        'Response': [response]
    })

    # Append the DataFrame to the CSV file
    if os.path.exists(response_csv_path):
        response_df.to_csv(response_csv_path, mode='a', header=False, index=False)
    else:
        response_df.to_csv(response_csv_path, mode='w', index=False)

    print(f"Prompt {i + 1} of {len(prompts)} processed")

print(f"All prompts processed. Responses saved to {response_csv_path}.")


Prompt 207 of 300 processed
Prompt 208 of 300 processed
Prompt 209 of 300 processed
Prompt 210 of 300 processed
Prompt 211 of 300 processed
Prompt 212 of 300 processed
Prompt 213 of 300 processed
Prompt 214 of 300 processed
Prompt 215 of 300 processed
Prompt 216 of 300 processed
Prompt 217 of 300 processed
Prompt 218 of 300 processed
Prompt 219 of 300 processed
Prompt 220 of 300 processed
Prompt 221 of 300 processed
Prompt 222 of 300 processed
Prompt 223 of 300 processed
Prompt 224 of 300 processed
Prompt 225 of 300 processed
Prompt 226 of 300 processed
Prompt 227 of 300 processed
Prompt 228 of 300 processed
Prompt 229 of 300 processed
Prompt 230 of 300 processed
Prompt 231 of 300 processed
Prompt 232 of 300 processed
Prompt 233 of 300 processed
Prompt 234 of 300 processed
Prompt 235 of 300 processed
Prompt 236 of 300 processed
Prompt 237 of 300 processed
Prompt 238 of 300 processed
Prompt 239 of 300 processed
Prompt 240 of 300 processed
Prompt 241 of 300 processed
Prompt 242 of 300 pr

In [11]:
response_csv_path = "/kaggle/input/responses-to-fix/gpt-j1x_responses.csv"
# Load the responses
df = pd.read_csv(response_csv_path)
response_csv_path = os.path.join("/kaggle/working/","gpt-j1x_responses_fix2.csv")
prompts = df['Prompt'].tolist()

# Iterate over the DataFrame
for i, row in df.iterrows():
    if pd.isnull(row['Response']):
        # Encode the prompt
        input_ids = tokenizer.encode(row['Prompt'], return_tensors="pt").to(device)

        # Generate a response
        # Generate a response
        output = gpt.generate(
                input_ids,
                attention_mask=torch.ones_like(input_ids),  # Set all positions to 1 (i.e., no padding)
                pad_token_id=tokenizer.eos_token_id,  # Use the EOS token as the PAD token
                do_sample=True,
                max_length=1024, 
            )

        # Calculate the number of tokens in the prompt
        prompt_length = input_ids.shape[-1]

        # Decode only the response, excluding the prompt
        response = tokenizer.decode(output[0, prompt_length:], skip_special_tokens=True)

        # Replace the NaN response with the new one
        df.at[i, 'Response'] = response
        
        response_df = pd.DataFrame({
        'Prompt': [prompts[i]],
        'Response': [response]
            })

              # Append the DataFrame to the CSV file
        if os.path.exists(response_csv_path):
            response_df.to_csv(response_csv_path, mode='a', header=False, index=False)
        else:
            response_df.to_csv(response_csv_path, mode='w', index=False)

        print(
            f"Regenerated response for prompt {i + 1} of {len(df)}. Updated responses saved to {response_csv_path}.")

print(f"All NaN responses regenerated. Updated responses saved to {response_csv_path}.")

Regenerated response for prompt 451 of 825. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
Regenerated response for prompt 475 of 825. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
Regenerated response for prompt 523 of 825. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
Regenerated response for prompt 649 of 825. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
Regenerated response for prompt 818 of 825. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
All NaN responses regenerated. Updated responses saved to /kaggle/working/gpt-j1x_responses_fix2.csv.
